In [ ]:
###############################################################################
# UPDATED
# Commented out reportdb button
# Added delay to sendreportfunc
# Redacted  
##############################################################################

from tkinter import *
from tkinter import filedialog # Need this for the file browser, isn't included in the "*" statement
from tkinter import messagebox
import tempfile
from PIL import Image
import pytesseract
import cv2
from datetime import *
import sys
import pyodbc
import csv
import pandas
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from twilio.rest import Client
from openpyxl import *
import time

# Defines LEAF branding HEX colors
LEAFPGreen = '#7AC143' #Main color, use for primary headlines and callouts
LEAFPDarkGreen = '#00703C' #used in LEAF logo and LEAF signature only
LEAFPMedGray = 'B9BAC1' #secondary color of the LEAF brand
LEAFPLightGray = 'F1F2F9' #secondary color of the LEAF brand
LEAFPBlack = '#000000' #secondary color of LEAF brand, never used for background

# Defines function for Exit command in menu bar
def quit():
    root.quit()
    root.destroy()
    exit()   

# Defines function for About command in menu bar
def About():
    messagebox.showinfo("Python Version", sys.version)
       
# Defines function for file browser    
def browsefunc():
    filename = filedialog.askopenfilename(initialdir="/",
    filetypes =(("PNG File", "*.png"),("BMP File", "*.bmp"),("JPEG File", "*.jpeg"),("TIFF File","*.tif")),
    title = "What Image Shall Argos Gaze Upon?"
                           ) 
    PathTextBox.delete("1.0",END)
    PathTextBox.insert(END,filename)

# Defines the function for the SendReportButton    
def sendreportfunc():
    reportdbfunc(),
    textfunc(),
    time.sleep(15),
    emailfunc()
    
# Defines the function to email report    
def emailfunc():
    fromaddr = "ArgosOCR@gmail.com"
    toaddr = ['jcorderman@leafnow.com', 'johncorderman@gmail.com']
    msg = MIMEMultipart()
    msg['From'] = fromaddr
    msg['To'] = ', '.join(toaddr)
    msg['Subject'] = "Argos OCR Results"
    body = "Please find the results attached."
    msg.attach(MIMEText(body, 'plain'))
    filename = "ArgosOCRData.xlsx"
    attachment = open("ArgosOCRReport.xlsx", "rb")
    part = MIMEBase('application', 'octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', "attachment; filename= %s" % filename)
    msg.attach(part)
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(fromaddr, "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
    text = msg.as_string()
    server.sendmail(fromaddr, toaddr, text)
    server.quit()

def textfunc():
    account_sid = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' # from twilio.com/console
    auth_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXx' # from twilio.com/console
    client = Client(account_sid, auth_token)
    num=['+16609999999','+16608888888'] # destination numbers

    for number in num: 
        message = client.messages.create(
            to=number, 
            from_='+12679999999', # twilio purchased phone number
            body="Argos OCR results will be emailed in a moment.  Please check your inbox.")
    
# Defines function for writing results to DB
def insertdbfunc():
    connection = pyodbc.connect(
    r'DRIVER={ODBC Driver 13 for SQL Server};'
    r'SERVER=HP\SQLSERVEREXPRESS;'
    r'DATABASE=ArgosTestDB;'
    r'Trusted_Connection=yes;'
    )
    cursor = connection.cursor()
    # This execute statement is sensitive.  Some get's require the interval "()" to be populated, the ocrtype doesnt for some reason
    # Must use the same number of ?'s as columns that are being added to
    # "end" in the gets appear to be case sensitive
    # Having trouble both getting a current datestamp, and converting the hh:mm:ss.ms that is captured into the table
    cursor.execute("""INSERT INTO dbo.OCR_Results(Date, Document, OCRFunction, Results, TimeElapsed) 
    VALUES (?, ?, ?, ?, ?)""", (datetime.now(), PathTextBox.get('1.0','end-1c'), 
    ocrtype.get(), OCROutputTextBox.get('1.0','end'), TimeTextBox.get('1.0','end-1c'))) 
    connection.commit()

# Defines function for reporting on SQL table    
def reportdbfunc():
    connection = pyodbc.connect(
    r'DRIVER={ODBC Driver 13 for SQL Server};'
    r'SERVER=HP\SQLSERVEREXPRESS;'
    r'DATABASE=ArgosTestDB;'
    r'Trusted_Connection=yes;'
    )
    cursor = connection.cursor()
    # Have to convert OCRFunction from text to varchar.  The db is somehow storing this column as varchar even though I have set to text.
    # You can't "=" text to varchar so have to convert it 
    cursor.execute("SELECT * FROM dbo.OCR_Results")
    row = cursor.fetchall()
    with open('ArgosOCRData.csv', 'w', newline='') as csvfile:
        a = csv.writer(csvfile, delimiter=',')
        a.writerows(row)
            
    column_names = ['Date','Document','OCRFunction','Results','TimeElapsed']

    # wrong encoding type may throw errors so can try different types like
    # 'ISO-8859-1' or 'utf-8' or 'cp1252'
    pandas.read_csv('ArgosOCRData.csv', encoding='ISO-8859-1', names = column_names).to_excel('ArgosOCRData.xlsx')
        
    wb = load_workbook('ArgosOCRData.xlsx')
    ws = wb.active
    c = ws['B2']
    # Freezes panes above and to the left of the cell noted in line above
    ws.freeze_panes = c
    # Deletes the 1st column
    ws.delete_cols(1)

    #This statement evaluates widest cell in column and adjusts the column to the max width
    for col in ws.columns:
         max_length = 0
         column = col[0].column # Get the column name
         for cell in col:
             try: # Necessary to avoid error on empty cells
                 if len(str(cell.value)) > max_length:
                     max_length = len(cell.value)
             except:
                 pass
         adjusted_width = (max_length + 2) #* 1.2
         ws.column_dimensions[column].width = adjusted_width
        
    wb.save('ArgosOCRReport.xlsx')     
    
# Defines the OCR function
# This function is operational but verbose, needs cleanup.  Can move individual if/elif variables into the function variables so that they're not repeated in every if statement
def OCRfunc():
    # Commences stopwatch for this function
    start_time = datetime.now() 
    # Defines two variables using text get for this function (funtion only, non-global), the "end-1c" gets rid of text on the end of the filepath
    path = PathTextBox.get('1.0','end-1c')
    ocr = ocrtype.get()
    if path and ocr == "Payment Amount":    
        # Defines the path image as a variable
        image = cv2.imread(path)
        # Crops image to defined area
        cropped = image[330:560, 185:560]
        # Defines the cropped area as a variable
        img = Image.fromarray(cropped) 
        # Defines the ocr string as a variable
        Text = (pytesseract.image_to_string(img))
        # Parses the previous variable for characters between "@" and "("
        Substring=Text[Text.find("@")+3:Text.find("(")]
        # Deletes any text in the output text box and inserts the ocr findings
        OCROutputTextBox.delete('1.0',END)
        OCROutputTextBox.insert(END,Substring)
        # Ends stopwatch to calculate elapsed time of the function and sends to text box
        time_elapsed = datetime.now() - start_time
        TimeTextBox.delete('1.0', END)
        TimeTextBox.insert(END, format(time_elapsed))
        # Calls the function to send the results to the database
        insertdbfunc()
    elif path and ocr == "Payment Region":    
        # Defines the path image as a variable
        image = cv2.imread(path)
        # Crops image to defined area
        cropped = image[330:560, 185:560]
        # Defines the cropped area as a variable
        img = Image.fromarray(cropped) 
        # Next three lines resize the cropped region; without resize preprocessing, "Payment" is recognized as "Pavment"
        r = 1200.0 / cropped.shape[1]
        dim = (1200, int(cropped.shape[0] * r))
        resized = cv2.resize(cropped, dim, interpolation = cv2.INTER_AREA)
        # Defines the ocr string as a variable
        Text = (pytesseract.image_to_string(resized))
        # Deletes any text in the output text box and inserts the ocr findings
        OCROutputTextBox.delete('1.0',END)
        OCROutputTextBox.insert(END,Text)
        # Ends stopwatch to calculate elapsed time of the function and sends to text box
        time_elapsed = datetime.now() - start_time
        TimeTextBox.delete('1.0', END)
        TimeTextBox.insert(END, format(time_elapsed))
        # Calls the function to send the results to the database
        insertdbfunc()
    elif path and ocr == "Term":    
        # Defines the path image as a variable
        image = cv2.imread(path)
        # Crops image to defined area
        cropped = image[339:560, 54:192]
        # Defines the cropped area as a variable
        img = Image.fromarray(cropped) 
        # Next three lines resize the cropped region
        r = 1200.0 / cropped.shape[1]
        dim = (1200, int(cropped.shape[0] * r))
        resized = cv2.resize(cropped, dim, interpolation = cv2.INTER_AREA)
        # Defines the ocr string as a variable
        Text = (pytesseract.image_to_string(resized))
        # Selects substring from ":" to end, could also use .rsplit(':', 1)[0] to capture text from : to the beginning of the string 
        Parsed = Text.rsplit('S',1)[1]
        # Deletes any text in the output text box and inserts the ocr findings
        OCROutputTextBox.delete('1.0',END)
        # The strip suffix strips out leading spaces but is also supposed to strip out leading spaces, not sure if it's impacted by the rsplit above
        OCROutputTextBox.insert(END,Parsed.strip())
        # Ends stopwatch to calculate elapsed time of the function and sends to text box
        time_elapsed = datetime.now() - start_time
        TimeTextBox.delete('1.0', END)
        TimeTextBox.insert(END, format(time_elapsed)) 
        insertdbfunc()
    elif path and ocr == "Legal Name":    
        # Defines the path image as a variable
        image = cv2.imread(path)
        # Crops image to defined area
        cropped = image[132:177, 54:1092]
        # Defines the cropped area as a variable
        img = Image.fromarray(cropped) 
        # Next three lines resize the cropped region
        r = 1200.0 / cropped.shape[1]
        dim = (1200, int(cropped.shape[0] * r))
        resized = cv2.resize(cropped, dim, interpolation = cv2.INTER_AREA)
        # Defines the ocr string as a variable
        Text = (pytesseract.image_to_string(resized))
        # Selects substring from ":" to end, could also use .rsplit(':', 1)[0] to capture text from : to the beginning of the string 
        Parsed = Text.rsplit(': ',1)[1]
        # Deletes any text in the output text box and inserts the ocr findings
        OCROutputTextBox.delete('1.0',END)
        OCROutputTextBox.insert(END,Parsed)
        # Ends stopwatch to calculate elapsed time of the function and sends to text box
        time_elapsed = datetime.now() - start_time
        TimeTextBox.delete('1.0', END)
        TimeTextBox.insert(END, format(time_elapsed)) 
        insertdbfunc()
    elif path and ocr == "TIN":    
        # Defines the path image as a variable
        image = cv2.imread(path)
        # Crops image to defined area
        cropped = image[126:180, 1092:1335]
        # Defines the cropped area as a variable
        img = Image.fromarray(cropped) 
        # Next three lines resize the cropped region
        r = 1500.0 / cropped.shape[1]
        dim = (1500, int(cropped.shape[0] * r))
        resized = cv2.resize(cropped, dim, interpolation = cv2.INTER_AREA)
        # Defines the ocr string as a variable
        Text = (pytesseract.image_to_string(resized))
        # Selects substring from ":" to end, could also use .rsplit(':', 1)[0] to capture text from : to the beginning of the string 
        Parsed = Text.rsplit(':',1)[1]
        # Deletes any text in the output text box and inserts the ocr findings
        OCROutputTextBox.delete('1.0',END)
        OCROutputTextBox.insert(END,Parsed.strip())
        # Ends stopwatch to calculate elapsed time of the function and sends to text box
        time_elapsed = datetime.now() - start_time
        TimeTextBox.delete('1.0', END)
        TimeTextBox.insert(END, format(time_elapsed)) 
        insertdbfunc()
    elif path and ocr == "Number of Payments":    
        # Defines the path image as a variable
        image = cv2.imread(path)
        # Crops image to defined area
        cropped = image[330:560, 185:560]
        # Defines the cropped area as a variable
        img = Image.fromarray(cropped) 
        r = 1200.0 / cropped.shape[1]
        dim = (1200, int(cropped.shape[0] * r))
        resized = cv2.resize(cropped, dim, interpolation = cv2.INTER_AREA)
        # Defines the ocr string as a variable
        Text = (pytesseract.image_to_string(resized))
        # Parses the previous variable for characters between "@" and "("
        Substring=Text[Text.find("S")+13:Text.find("@")]
        # Deletes any text in the output text box and inserts the ocr findings
        OCROutputTextBox.delete('1.0',END)
        OCROutputTextBox.insert(END,Substring)
        # Ends stopwatch to calculate elapsed time of the function and sends to text box
        time_elapsed = datetime.now() - start_time
        TimeTextBox.delete('1.0', END)
        TimeTextBox.insert(END, format(time_elapsed))
        insertdbfunc()
    
    else:
        OCROutputTextBox.delete('1.0',END)
        TimeTextBox.delete('1.0',END)
        OCROutputTextBox.insert(END,"Please Browse and/or Focus")
    
# Following lines create the highest-level window named root
root =Tk()
root.title('Argos v.1.4 beta')
root.geometry("550x300")
root.iconbitmap("ArgosLogoFinal.ico")

# Insert a menu bar on the root window
menubar = Menu(root)
root.config(menu=menubar)

# Create a menu with menu cascades labeled "File" and "About) that brings up a menu, removes tearoff menu capability
filemenu = Menu(menubar, tearoff=0)
filemenu.add_command(label="Exit", command=quit)
menubar.add_cascade(label='File', menu=filemenu)

# Would need to add a function for a popup about window (define it about like the quit function and call the command below)
helpmenu = Menu(menubar, tearoff=0)
helpmenu.add_command(label="About", command=About)
menubar.add_cascade(label='Help', menu=helpmenu)

# Variable for the ocroption dropdown optionmenu.  The optionmenu class has no get function so you have to define a variable for it, which is done here.
ocrtype = StringVar(root)
ocrtype.set("Focus")

# Button for browsefunc
browsebutton = Button(root, text="Browse", command=browsefunc)
browsebutton.grid(row = 0, column=1, padx=2, pady=4, sticky="w"+"e")

# Text box for the selected file path 
PathTextBox = Text(root, height=1)
PathTextBox.grid(row=0, column=2, padx=2, pady=4)

# Dropdown list for function
OCROption = OptionMenu(root, ocrtype, "Payment Region", "Payment Amount", "Number of Payments", "Term", "Legal Name", "TIN")
OCROption.grid(row=1, column=1, padx=2, pady=4, sticky="w"+"e")

# Button for OCR function
OCRButton = Button(root,text="Gaze",command = OCRfunc)
OCRButton.grid(row=2, column=1, padx=2, pady=4, sticky="w"+"e")

# Text box for OCR output
OCROutputTextBox = Text(root, height=5)
OCROutputTextBox.grid(row=2, column=2, padx=2, pady=4)

#Label for elapsed time text output
TimeLabel = Label(root, text="Elapsed Time (hh:mm:ss.ms)")
TimeLabel.grid(row=3, column=1, padx=2, pady=4)

# Text box for elapsed time output
TimeTextBox = Text(root, height=1)
TimeTextBox.grid(row=3, column=2, padx=2, pady=4)

##########################################################################################################
# Button for OCR function.....................can probably be removed now unless there's a need to keep a manual option
#OCRButton = Button(root,text="Test InsertDBFunc",command = insertdbfunc)
#OCRButton.grid(row=4, column=1, padx=2, pady=4, sticky="w"+"e")

#############################################################################################

# Button for sendreportfunc
SendReportButton = Button(root,text="Report",command = sendreportfunc)
SendReportButton.grid(row=5, column=1, padx=2, pady=4, sticky="w"+"e")

# Statement runs the root window on a continuous loop, without this statement the window will appear for only about a millisecond so we need to run it on a continous loop to view it full time
root.mainloop ( )